<a href="https://colab.research.google.com/github/ADMITO-MITO/Treino-google-colab/blob/main/Dataset_hibrido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar dependências
!pip install ultralytics -q
!pip install roboflow -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 19.4 MB/s eta 0:00:00


In [5]:
# =============================================================================
# NOTEBOOK GOOGLE COLAB - TREINAMENTO YOLO COM DATASET HÍBRIDO
# =============================================================================
#
# INSTRUÇÕES:
# 1. Faça upload do arquivo "hybrid_dataset_for_colab.zip" para seu Google Drive
# 2. Execute as células na ordem
# 3. Aguarde o treinamento (2-4 horas)
# 4. Baixe o modelo treinado

# =============================================================================
# 1. CONFIGURAÇÃO INICIAL
# =============================================================================

# Verificar GPU
!nvidia-smi


# Imports
import os
import zipfile
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import Image, display
import torch
from ultralytics import YOLO

print("🚀 Ambiente configurado!")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


Fri Sep 12 02:14:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:

# =============================================================================
# 2. CONECTAR GOOGLE DRIVE
# =============================================================================

from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive conectado!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive conectado!


In [ ]:
# =============================================================================
# 3. COPIAR DATASET DA PASTA (SUBSTITUA A SEÇÃO DE EXTRAÇÃO)
# =============================================================================

# AJUSTE APENAS ESTA LINHA:
source_folder = "/content/drive/MyDrive/NOME_DA_SUA_PASTA"  # ← MUDE AQUI

# Resto do código (não mude):
import shutil, os

if not os.path.exists(source_folder):
    print("❌ Pasta não encontrada!")
    print("\n📁 Pastas disponíveis:")
    !ls "/content/drive/MyDrive/"
else:
    dataset_dir = "/content/dataset"
    if os.path.exists(dataset_dir):
        shutil.rmtree(dataset_dir)

    shutil.copytree(source_folder, dataset_dir)
    print("✅ Dataset copiado!")

    # Verificar estrutura
    !ls -la /content/dataset/

In [ ]:

# =============================================================================
# 4. VERIFICAR ESTRUTURA DO DATASET
# =============================================================================

print("📁 Verificando estrutura do dataset...")

# Listar estrutura
!ls -la /content/dataset/

# Verificar se tem as pastas necessárias
required_folders = [
    "/content/dataset/images/train",
    "/content/dataset/images/val",
    "/content/dataset/labels/train",
    "/content/dataset/labels/val"
]

all_good = True
for folder in required_folders:
    if os.path.exists(folder):
        count = len(os.listdir(folder))
        print(f"✅ {folder}: {count} arquivos")
    else:
        print(f"❌ {folder}: NÃO ENCONTRADA")
        all_good = False

if not all_good:
    print("\n🔍 ESTRUTURA ATUAL:")
    !find /content/dataset -type d | head -10

    print("\n💡 POSSÍVEIS SOLUÇÕES:")
    print("1. Verifique se copiou a pasta certa")
    print("2. A estrutura deve ser:")
    print("   dataset/")
    print("   ├── images/")
    print("   │   ├── train/")
    print("   │   └── val/")
    print("   ├── labels/")
    print("   │   ├── train/")
    print("   │   └── val/")
    print("   └── dataset.yaml")


In [ ]:

# =============================================================================
# 5. VERIFICAR ARQUIVO DE CONFIGURAÇÃO
# =============================================================================

# Procurar arquivo de configuração
config_files = []
for root, dirs, files in os.walk("/content/dataset"):
    for file in files:
        if file.endswith('.yaml') or file.endswith('.yml'):
            config_files.append(os.path.join(root, file))

if config_files:
    config_path = config_files[0]  # Usar o primeiro encontrado
    print(f"📋 Arquivo de configuração encontrado: {config_path}")

    # Carregar configuração
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)

    print(f"Classes: {config['nc']}")
    print("Nomes das classes:")
    for i, name in enumerate(config['names']):
        print(f"  {i:2d}. {name}")

    # Atualizar caminho no config
    config['path'] = '/content/dataset'
    with open(config_path, 'w') as f:
        yaml.dump(config, f)

else:
    print("❌ Arquivo .yaml não encontrado!")
    print("Arquivos disponíveis:")
    !find /content/dataset -name "*.yaml" -o -name "*.yml"



In [ ]:

# =============================================================================
# 6. CONTAR IMAGENS
# =============================================================================

if os.path.exists("/content/dataset/images/train"):
    train_imgs = len([f for f in os.listdir("/content/dataset/images/train")
                     if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    print(f"📊 Imagens de treino: {train_imgs}")
else:
    train_imgs = 0

if os.path.exists("/content/dataset/images/val"):
    val_imgs = len([f for f in os.listdir("/content/dataset/images/val")
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    print(f"📊 Imagens de validação: {val_imgs}")
else:
    val_imgs = 0

total_imgs = train_imgs + val_imgs
print(f"🎯 Total de imagens: {total_imgs}")

if total_imgs < 100:
    print("⚠️ ATENÇÃO: Poucas imagens encontradas!")
    print("Verifique se copiou a pasta correta")



In [ ]:

# =============================================================================
# 7. VISUALIZAR AMOSTRAS (se tudo estiver OK)
# =============================================================================

if all_good and total_imgs > 0:
    import random
    import cv2
    from PIL import Image as PILImage
    import numpy as np

    print("🖼️ AMOSTRAS DO DATASET:")

    def show_sample_images(dataset_path, num_samples=4):
        img_dir = Path(dataset_path) / "images" / "train"
        lbl_dir = Path(dataset_path) / "labels" / "train"

        img_files = [f for f in img_dir.glob("*") if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]

        if len(img_files) == 0:
            print("❌ Nenhuma imagem encontrada!")
            return

        samples = random.sample(img_files, min(num_samples, len(img_files)))

        plt.figure(figsize=(12, 8))

        for i, img_file in enumerate(samples):
            # Carregar imagem
            img = cv2.imread(str(img_file))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            h, w = img.shape[:2]

            # Carregar anotações
            lbl_file = lbl_dir / f"{img_file.stem}.txt"
            if lbl_file.exists():
                with open(lbl_file, 'r') as f:
                    for line in f:
                        if line.strip():
                            parts = line.strip().split()
                            class_id = int(parts[0])
                            x_center, y_center, width, height = map(float, parts[1:5])

                            # Converter para coordenadas da imagem
                            x1 = int((x_center - width/2) * w)
                            y1 = int((y_center - height/2) * h)
                            x2 = int((x_center + width/2) * w)
                            y2 = int((y_center + height/2) * h)

                            # Desenhar bbox
                            color = (255, 0, 0) if i % 2 == 0 else (0, 255, 0)
                            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

                            # Adicionar label se temos config
                            if 'config' in locals() and class_id < len(config['names']):
                                class_name = config['names'][class_id]
                                cv2.putText(img, class_name, (x1, y1-10),
                                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

            plt.subplot(2, 2, i+1)
            plt.imshow(img)
            plt.title(f"Amostra {i+1}")
            plt.axis('off')

        plt.tight_layout()
        plt.show()

    show_sample_images("/content/dataset")
else:
    print("⚠️ Pulando visualização - corrija os problemas acima primeiro")


In [ ]:
# =============================================================================
# 8. CONFIGURAÇÃO DE TREINAMENTO
# =============================================================================

if 'config_path' in locals() and all_good:
    print("⚙️ DATASET PRONTO PARA TREINAMENTO!")
    print(f"📋 Arquivo de config: {config_path}")
    print(f"🎯 Total de imagens: {total_imgs}")

    # Parâmetros de treinamento
    EPOCHS = 150
    MODEL_SIZE = "s"
    BATCH_SIZE = 16

    print(f"⚙️ CONFIGURAÇÕES:")
    print(f"Épocas: {EPOCHS}")
    print(f"Modelo: YOLOv8{MODEL_SIZE}")
    print(f"Batch size: {BATCH_SIZE}")

else:
    print("❌ DATASET COM PROBLEMAS - corrija antes de prosseguir")
    config_path = None

print("\n✅ CÉLULA CONCLUÍDA!")
print("Se tudo estiver OK, pode prosseguir para o treinamento")


In [ ]:

# =============================================================================
# 7. TREINAMENTO
# =============================================================================

print("🚀 INICIANDO TREINAMENTO...")
print("="*50)

# Carregar modelo pré-treinado
model = YOLO(f'yolov8{MODEL_SIZE}.pt')

# Treinamento com configurações otimizadas para Colab
results = model.train(
    data=config_path,
    epochs=EPOCHS,
    imgsz=640,
    batch=BATCH_SIZE,
    device=0,  # GPU

    # Otimizações para Colab
    cache=True,
    workers=2,  # Colab tem limitações

    # Configurações de aprendizado
    optimizer='AdamW',
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,

    # Early stopping
    patience=30,

    # Data augmentation
    augment=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.1,
    scale=0.5,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,

    # Salvar configurações
    save=True,
    save_period=0,

    # Nome do experimento
    name="hybrid_yolo_training",

    # Logs
    verbose=True,
    plots=True
)

print("✅ TREINAMENTO CONCLUÍDO!")


In [ ]:

# =============================================================================
# 8. AVALIAÇÃO DOS RESULTADOS
# =============================================================================

print("📊 AVALIANDO MODELO...")

# Carregar melhor modelo
best_model_path = results.save_dir / "weights" / "best.pt"
best_model = YOLO(str(best_model_path))

# Validação
val_results = best_model.val(data=config_path)

print("📈 RESULTADOS FINAIS:")
print(f"mAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")
print(f"Precisão: {val_results.box.mp:.4f}")
print(f"Recall: {val_results.box.mr:.4f}")

# Mostrar métricas por classe
if hasattr(val_results.box, 'ap_class_index'):
    print("\n📋 DESEMPENHO POR CLASSE:")
    for i, class_idx in enumerate(val_results.box.ap_class_index):
        if i < len(val_results.box.ap50):
            class_name = config['names'][class_idx]
            ap50 = val_results.box.ap50[i]
            print(f"  {class_name:<15}: mAP50 = {ap50:.3f}")


In [ ]:

# =============================================================================
# 9. VISUALIZAR GRÁFICOS DE TREINAMENTO
# =============================================================================

# Mostrar gráficos de treinamento
results_dir = results.save_dir

# Listar arquivos de resultado
print("📁 Arquivos de resultado:")
!ls -la {results_dir}

# Mostrar gráficos se existirem
plot_files = [
    "results.png",
    "confusion_matrix.png",
    "F1_curve.png",
    "P_curve.png",
    "R_curve.png",
    "PR_curve.png"
]

for plot_file in plot_files:
    plot_path = results_dir / plot_file
    if plot_path.exists():
        print(f"\n📊 {plot_file.replace('.png', '').replace('_', ' ').title()}:")
        display(Image(str(plot_path)))


In [ ]:

# =============================================================================
# 10. TESTAR O MODELO
# =============================================================================

print("🧪 TESTANDO O MODELO...")

# Testar em algumas imagens de validação
val_img_dir = Path("/content/dataset/images/val")
test_images = list(val_img_dir.glob("*.jpg"))[:5]  # 5 imagens de teste

plt.figure(figsize=(20, 4))
for i, img_path in enumerate(test_images):
    # Fazer predição
    results_pred = best_model(str(img_path))

    # Plotar resultado
    plt.subplot(1, 5, i+1)

    # Carregar imagem original
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Desenhar predições
    if len(results_pred[0].boxes) > 0:
        for box in results_pred[0].boxes:
            # Coordenadas
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            conf = box.conf[0].cpu().numpy()
            cls = int(box.cls[0].cpu().numpy())

            if conf > 0.5:  # Filtrar baixa confiança
                # Desenhar bbox
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Label
                label = f"{config['names'][cls]}: {conf:.2f}"
                cv2.putText(img, label, (x1, y1-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    plt.imshow(img)
    plt.title(f"Teste {i+1}")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:

# =============================================================================
# 11. EXPORTAR MODELO
# =============================================================================

print("📦 EXPORTANDO MODELO...")

# Exportar para ONNX (formato otimizado)
try:
    onnx_path = best_model.export(format='onnx')
    print(f"✅ ONNX exportado: {onnx_path}")
except Exception as e:
    print(f"⚠️ Erro ao exportar ONNX: {e}")

# Exportar para TorchScript
try:
    torchscript_path = best_model.export(format='torchscript')
    print(f"✅ TorchScript exportado: {torchscript_path}")
except Exception as e:
    print(f"⚠️ Erro ao exportar TorchScript: {e}")


In [ ]:
print("💾 SALVANDO APENAS O MODELO NO GOOGLE DRIVE...")

import shutil
import os

# Criar pasta no Drive
drive_results_dir = "/content/drive/MyDrive/YOLO_Results"
os.makedirs(drive_results_dir, exist_ok=True)

# Copiar APENAS o best.pt
best_model_path = results.save_dir / "weights" / "best.pt"
dest_path = f"{drive_results_dir}/modelo_treinado.pt"

if best_model_path.exists():
    shutil.copy2(best_model_path, dest_path)
    print(f"✅ Modelo salvo: {dest_path}")

    # Mostrar tamanho do arquivo
    size_mb = os.path.getsize(dest_path) / (1024 * 1024)
    print(f"📏 Tamanho: {size_mb:.1f} MB")
else:
    print("❌ Modelo não encontrado!")

print(f"📁 Arquivo salvo em: {drive_results_dir}/modelo_treinado.pt")

In [ ]:

# =============================================================================
# 13. RELATÓRIO FINAL
# =============================================================================

print("\n" + "="*60)
print("🎉 TREINAMENTO CONCLUÍDO COM SUCESSO!")
print("="*60)

print(f"📊 RESUMO DOS RESULTADOS:")
print(f"  • mAP50: {val_results.box.map50:.3f}")
print(f"  • mAP50-95: {val_results.box.map:.3f}")
print(f"  • Precisão média: {val_results.box.mp:.3f}")
print(f"  • Recall médio: {val_results.box.mr:.3f}")

print(f"\n📁 ARQUIVOS PRINCIPAIS:")
print(f"  • Modelo treinado: {drive_results_dir}/modelo_treinado.pt")
print(f"  • Gráficos: {drive_results_dir}/graficos_treinamento.png")
print(f"  • Matriz confusão: {drive_results_dir}/matriz_confusao.png")

print(f"\n🚀 PRÓXIMOS PASSOS:")
print("1. Baixe o arquivo 'modelo_treinado.pt' do Google Drive")
print("2. Use este modelo no seu código Python local")
print("3. Substitua o caminho no seu código original:")
print("   modelo = YOLO('caminho/para/modelo_treinado.pt')")

print(f"\n📋 CÓDIGO PARA USO LOCAL:")
print("```python")
print("from ultralytics import YOLO")
print("import cv2")
print("")
print("# Carregar modelo treinado")
print("modelo = YOLO('modelo_treinado.pt')")
print("")
print("# Classes do seu modelo")
print(f"classes = {config['names']}")
print("")
print("# Usar em webcam")
print("cap = cv2.VideoCapture(0)")
print("while True:")
print("    ret, frame = cap.read()")
print("    if ret:")
print("        results = modelo(frame)")
print("        # Processar resultados...")
print("```")


In [ ]:

# =============================================================================
# 14. FUNÇÃO DE DOWNLOAD (OPCIONAL)
# =============================================================================

def download_from_colab(file_path, download_name=None):
    """Função para forçar download de arquivo do Colab"""
    from google.colab import files

    if download_name is None:
        download_name = Path(file_path).name

    try:
        files.download(file_path)
        print(f"📥 Download iniciado: {download_name}")
    except Exception as e:
        print(f"❌ Erro no download: {e}")
        print("💡 Use o Google Drive para acessar os arquivos")

# Descomente para forçar download do modelo principal
# download_from_colab(str(results_dir / "weights" / "best.pt"), "modelo_treinado.pt")

print("\n✅ NOTEBOOK CONCLUÍDO!")
print("🎯 Seu modelo está pronto para uso!")


In [ ]:

# =============================================================================
# CÓDIGO BONUS: TESTE INTERATIVO
# =============================================================================

# Função para teste interativo (descomente para usar)
"""
def test_interactive():
    from google.colab import files
    import io
    from PIL import Image

    print("📤 Faça upload de uma imagem para testar o modelo:")
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"🧪 Testando: {filename}")

        # Carregar imagem
        img = Image.open(io.BytesIO(uploaded[filename]))
        img = img.convert('RGB')

        # Fazer predição
        results = best_model(img)

        # Mostrar resultado
        results[0].show()

        # Imprimir detecções
        if len(results[0].boxes) > 0:
            print("🎯 Objetos detectados:")
            for box in results[0].boxes:
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                class_name = config['names'][cls]
                print(f"  • {class_name}: {conf:.2f}")
        else:
            print("❌ Nenhum objeto detectado")

# Descomente para ativar teste interativo
# test_interactive()
"""